### Extra: Mathematical model of an infectious disease

Consider the [SIR model](https://de.wikipedia.org/wiki/SIR-Modell) characterized through the following coupled systems of ordinary differential equations:
* ${\frac {dS}{dt}}=-c{\frac {S}{N}}I$
* ${\frac {dI}{dt}}=c{\frac {S}{N}}I-wI$
* ${\frac {dR}{dt}}=wI$

with the boundary condition that the total number of individuals N is the sum of S, I, and R and constant
* $N=S+I+R$

(Obviously, the boundary condition is trivially fulfilled as the sum of the derivatives is zero, i.e. no births or deaths are included in this version of the model.)

Herein are:
* S: number of susceptible individuals
* I: number of infected individuals
* R: number of recovered (removed) individuals
* c: probability of disease transmission
* w: rate of recovery
* t: time (independent variable)

Your tasks:
* Solve this system of equations using methods presented in the course and plot S, I and R as a function of time.
* Change the parameters c and w and study their impact.
* Make the model more dynamic by e.g. introducing a birth and a death rate, or vaccination.

The SIR model is one of the simplest of a group of [compartmental models](https://en.wikipedia.org/wiki/Compartmental_models_in_epidemiology) in theoretical biology. The basic reproduction number $R$ is given by $R=c/w$. It describes the expected number of new infections caused by one infected individual (in a population where all subjects are susceptible).
